In [1]:
import pandas as pd                        
from pytrends.request import TrendReq
pytrend = TrendReq()
import regex as re
from collections import Counter
from bs4 import BeautifulSoup
from lxml import etree
import requests
from warnings import filterwarnings
filterwarnings("ignore")



In [22]:
import tweepy as tp
from datetime import datetime, timedelta

bearerToken = "AAAAAAAAAAAAAAAAAAAAAOUBZwEAAAAAuSI9Lk9VJF5p8oZ60%2Ffnb25FSXo%3DsH2SwTWEqpQOe0acAUZeAiPdazuwZYetImYMSn9Wzk7dmXR1VV"

client = tp.Client(bearer_token=bearerToken, wait_on_rate_limit=True)

def convertDate(tweetObject):
    tweetDate = tweetObject['created_at']
    tweetDate = tweetDate.replace("T", " ")
    tweetDate = str(tweetDate.split(".")[0])
    tweetDate = datetime.strptime(tweetDate, '%Y-%m-%d %H:%M:%S')
    return(tweetDate)



def convertTime(inputDate):

    inputDateSplit = inputDate.split("-")

    year = int(inputDateSplit[0])
    month = int(inputDateSplit[1])
    day = int(inputDateSplit[2])

    if month < 10 and day < 10:
        timeString = f"{year}-0{month}-0{day}T00:00:00Z"
        return timeString
    elif month < 10 and day >= 10:
        timeString = f"{year}-0{month}-{day}T00:00:00Z"
        return timeString
    elif month >= 10 and day < 10:
        timeString = f"{year}-{month}-0{day}T00:00:00Z"
        return timeString
    else:
        timeString = f"{year}-{month}-{day}T00:00:00Z"
        return timeString

def generateTweets(date, df, brand):

    tweets = client.search_recent_tweets(query=f"{brand} -is:retweet lang:EN", tweet_fields=['context_annotations', 'created_at', 'text', 'author_id', 'source', 'public_metrics'], max_results=100, end_time=date)
        
    for tweet in tweets.data:
        tweet_ID = tweet.id
        tweet_content = tweet.text
        tweet_date = tweet.created_at
        tweet_retweets = tweet.public_metrics['retweet_count']
        tweet_replies = tweet.public_metrics['reply_count']
        tweet_likes = tweet.public_metrics['like_count']
        tweet_quotes = tweet.public_metrics['quote_count']

        df = df.append({'id': str(tweet_ID), 'full_text':str(tweet_content), 'created_at':tweet_date,  \
                'rt_count':tweet_retweets, 're_count':tweet_replies, 'like_count':tweet_likes, 'quotes':tweet_quotes}, ignore_index=True)


    return df

def filterItems(topTenList):
    filteredItems = []
    for item in topTenList:
    # itemToTest = topTenList[1]
        pytrend.build_payload([item], timeframe='today 1-m')
        relatedTopics = pytrend.related_topics()
        try:
            relatedTopicList = list(relatedTopics[item]['top']['topic_type'][:5])
            relatedTopicList = [x.lower() for x in relatedTopicList]
            if "food" in relatedTopicList or "drink" in relatedTopicList or "beverage" in relatedTopicList:
                filteredItems.append(item)
        except Exception as e:
            # print(e, item)
            None

    return filteredItems


def generateDF(phrase):
    df = pd.DataFrame(columns=['id', 'full_text', 'created_at', 'rt_count', 're_count', 'like_count', 'quotes'])

    
    for i in range(6):
        date = convertTime((datetime.now() - timedelta(days=i)).strftime("%Y-%m-%d"))
        
        df = generateTweets(date, df, phrase)

    df.drop_duplicates(subset="id", inplace=True)

    regexStart = str(phrase.split(" ")[-1])
    regexStart = regexStart.replace('"', '')

    df["trending_product"] = "null"
    for a, tweet in enumerate(df.full_text):
        regex = re.search(fr'{regexStart}(..*)', tweet)
        try:
            df["trending_product"][a] = regex.group(1)
        except AttributeError:
            df["trending_product"][a] = regex

    
    df['trending_product'] = df['trending_product'].apply(lambda x: " ".join(str(x).split(" ")[1:3]))
    df = df[df.trending_product.str.len() > 3]
    df['trending_product'] = df['trending_product'].apply(lambda x: str(x).lower())
    df['trending_product'] = df['trending_product'].apply(lambda x: str(x).replace("@", ""))

    return df

def generateTopTen(productList):

    topTen = Counter(productList).most_common(20)

    topTenList = []
    for item in topTen:
        if item != "null":
            topTenList.append(item[0])

    return(topTenList)

def googleTrendTopics(topList):
    filteredItems = []
    for item in topList:
    # itemToTest = topTenList[1]
        pytrend.build_payload([item], timeframe='today 1-m')
        relatedTopics = pytrend.related_topics()
        try:
            relatedTopicList = list(relatedTopics[item]['top']['topic_type'][:5])
            relatedTopicListLower = [x.lower() for x in relatedTopicList]
            relatedTopicListLower = list(relatedTopicListLower)
            if "food" in relatedTopicListLower or "drink" in relatedTopicListLower or "beverage" in relatedTopicListLower:
                filteredItems.append(item)
        except Exception as e:
            # print(e, item)
            None

    return(filteredItems)


def generateNews(brandName):
    urlBrandName = "+".join(brandName.split(" "))

    URL = f"https://news.google.com/search?q={urlBrandName}&hl=en-US&gl=US&ceid=US:en"
    
    HEADERS = ({'User-Agent':
                'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 \
                (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',\
                'Accept-Language': 'en-US, en;q=0.5'})
    
    webpage = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "html.parser")
    news = soup.find_all("a", class_="DY5T1d")

    for article in news:
        regex = r"href=\".(..*)\""

        articleText = article.text.lower()
        articleText = articleText.replace("'s", "")
        if brandName in articleText or "-".join(brandName.split(" ")) in articleText:
            # print(article.text.lower())
            url = re.findall(regex, str(article), re.MULTILINE)[0]
            return(articleText, url)


def generateTrendingProducts(useCustomDF=False, df1=None, df2=None):

    productList = []
    phrases = ['"tried the new"', '"has anyone tried"']

    if useCustomDF == False:
        for phrase in phrases:

            df_phrase = generateDF(phrase)
            productList = productList + list(df_phrase.trending_product)

    else:
        productList = list(df1.trending_product) + list(df2.trending_product)


    topTenList = generateTopTen(productList)

    filteredList = googleTrendTopics(topTenList)
    trendingArticleNames = []
    trendingURLNames = []
    for item in filteredList:
        try:
            trendingText, trendingURL = generateNews(item)
            trendingArticleNames.append(trendingText)

            newsUrl = f"https://news.google.com{trendingURL}"
            r = requests.get(newsUrl, allow_redirects=False)
            urlFix = r.headers['Location']
            r2 = requests.get(urlFix, allow_redirects=False)

            trendingURLNames.append(r2.headers['Location'])
        
        except:
            None

    return(trendingArticleNames, trendingURLNames)


    

In [35]:

dfTest1 = pd.read_excel("/Users/vetlefrantzvaag/Desktop/DashThesis/multipage/data/tried_new_20.05.xlsx", engine="openpyxl", index_col=[0])
dfTest2 = pd.read_excel("/Users/vetlefrantzvaag/Desktop/DashThesis/multipage/data/tried_anyone_20.05.xlsx", engine="openpyxl", index_col=[0])

articleName, urlName = generateTrendingProducts()

['mountain dew will be bringing back its typhoon flavor',
 'sweet, spicy and sour: john chantarasak’s recipes for thai salads',
 'sixpoint brewery & mike hot honey release slice sipper']

In [73]:
for i, article in enumerate(trendingArticles):
    print(f"#{i+1}: {article}")
    break

#1: hard mountain dew comes to minnesota


In [263]:
df

In [264]:
df_tried_anyone = df_tried_anyone[['id', 'full_text', 'trending_product']]

In [265]:
df_tried_anyone.to_excel("tried_anyone_20.05.xlsx")

In [12]:
#import the libraries
import pandas as pd                        
from pytrends.request import TrendReq
pytrend = TrendReq()

#provide your search terms
kw_list=['"Nitro pepsi"']

#search interest per region
#run model for keywords (can also be competitors)
pytrend.build_payload(kw_list, timeframe='today 1-m')

# Interest by Region
regiondf = pytrend.interest_by_region()
#looking at rows where all values are not equal to 0
regiondf = regiondf[(regiondf != 0).all(1)]

#drop all rows that have null values in all columns
regiondf.dropna(how='all',axis=0, inplace=True)

# #visualise
# regiondf.plot(figsize=(20, 12), y=kw_list, kind ='bar')
regiondf

,"""Nitro pepsi"""
geoName,
Australia,10
Canada,18
Mexico,4
Netherlands,2
Norway,7
Poland,8
South Korea,6
United Kingdom,4
United States,100


In [8]:
brand = "pepsi nitro"
brandSplit = brand.split(" ")

if len(brandSplit) > 1:
    brand = f'"{" ".join(brandSplit)}"'

print(brand)

"pepsi nitro"
